In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("upvote_train.csv")
df.head(5)

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [2]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df.Tag = le.fit_transform(df.Tag.astype(str))

In [4]:
df.skew()

ID            -0.000298
Tag            0.337230
Reputation    11.230027
Answers        3.380169
Username       0.142088
Views         14.370704
Upvotes       74.249689
dtype: float64

In [5]:
upper_Reputation = df['Reputation'].mean() + 1.5*df['Reputation'].std()
lower_Reputation = df['Reputation'].mean() - 1.5*df['Reputation'].std()
df.loc[df['Reputation']> 5118.0,'Reputation'] = 5119.0

upper_views = df['Views'].mean() + 1.5*df['Views'].std()
lower_views = df['Views'].mean() - 1.5*df['Views'].std()
df.loc[df['Views']> 26870.0,'Views'] = 26870.0

upper_answers = df['Answers'].mean() + 1.5*df['Answers'].std()
lower_answers = df['Answers'].mean() - 1.5*df['Answers'].std()
df.loc[df['Answers']> 5.0,'Answers'] = 5.0

upper_upvotes = df['Upvotes'].mean() + 1.5*df['Upvotes'].std()
lower_upvotes = df['Upvotes'].mean() - 1.5*df['Upvotes'].std()
df.loc[df['Upvotes']> 107.0,'Upvotes'] = 107.0

In [6]:
df.skew()

ID           -0.000298
Tag           0.337230
Reputation    0.498064
Answers      -0.001231
Username      0.142088
Views         0.357091
Upvotes       0.500881
dtype: float64

In [7]:
x = df.iloc[:,1:6]
y = df.iloc[:,-1]

In [8]:
from xgboost import XGBRegressor
xgbr = XGBRegressor()
xgbr.fit(x,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [9]:
xgb_pred = xgbr.predict(x)

In [15]:
df["Predections"] = xgb_pred

In [10]:
r2 = xgbr.score(x,y)
r2

0.7789310535927079

In [11]:
adj_r2 = 1 - (1-xgbr.score(x,y))*(len(y)-1)/(len(y)-x.shape[1]-1)
adj_r2

0.7789277044590236

In [17]:
from sklearn import metrics
MSE_ln = metrics.mean_squared_error(y,xgb_pred)
MSE_ln

388.7276701358428

In [18]:
error = y - xgb_pred
error_ln = np.abs(error)

In [19]:
MAPE_ln = np.mean(error_ln/xgb_pred)*100
MAPE_ln

51.0106577104562

In [20]:
acc = 100 - MAPE_ln
acc

48.9893422895438

In [21]:
df_main = pd.read_csv("upvote_test.csv")
df_main

,ID,Tag,Reputation,Answers,Username,Views
0,366953,a,5645.0,3.0,50652,33200.0
1,71864,c,24511.0,6.0,37685,2730.0
2,141692,i,927.0,1.0,135293,21167.0
3,316833,i,21.0,6.0,166998,18528.0
4,440445,i,4475.0,10.0,53504,57240.0
...,...,...,...,...,...,...
141443,47187,j,4681.0,4.0,152060,2469.0
141444,329126,j,7782.0,3.0,89235,18998.0
141445,282334,p,435.0,5.0,119788,57048.0
141446,386629,o,176.0,3.0,86824,10193.0


In [22]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_main.Tag = le.fit_transform(df_main.Tag.astype(str))

In [23]:
df_main.skew()

ID             0.000693
Tag            0.332509
Reputation    11.318536
Answers        3.398091
Username       0.147448
Views         13.532848
dtype: float64

In [24]:
df_main.describe()

,ID,Tag,Reputation,Answers,Username,Views
count,141448.000000,141448.000000,1.414480e+05,141448.000000,141448.000000,1.414480e+05
mean,235743.073497,3.552542,7.920927e+03,3.914873,81348.231117,2.984633e+04
std,136269.867118,2.494553,2.791072e+04,3.577460,49046.098215,8.034374e+04
min,7.000000,0.000000,0.000000e+00,0.000000,4.000000,9.000000e+00
25%,117797.000000,1.000000,2.860000e+02,2.000000,40222.750000,2.608000e+03
50%,235830.000000,4.000000,1.245000e+03,3.000000,78795.500000,8.977000e+03
75%,353616.000000,6.000000,5.123000e+03,5.000000,122149.000000,2.698925e+04
max,471488.000000,9.000000,1.042428e+06,73.000000,175737.000000,5.004669e+06


In [25]:
upper_Reputation = df_main['Reputation'].mean() + 1.5*df_main['Reputation'].std()
lower_Reputation = df_main['Reputation'].mean() - 1.5*df_main['Reputation'].std()
df_main.loc[df_main['Reputation']> 5123.0,'Reputation'] = 5123.0

upper = df_main['Views'].mean() + 1.5*df_main['Views'].std()
lower = df_main['Views'].mean() - 1.5*df_main['Views'].std()
df_main.loc[df_main['Views']> 26989.0,'Views'] = 26989.0

upper = df_main['Answers'].mean() + 1.5*df_main['Answers'].std()
lower = df_main['Answers'].mean() - 1.5*df_main['Answers'].std()
df_main.loc[df_main['Answers']> 5.0,'Answers'] = 5.0

In [26]:
x_ = df_main.iloc[:,1:6]
x_

,Tag,Reputation,Answers,Username,Views
0,0,5123.0,3.0,50652,26989.0
1,1,5123.0,5.0,37685,2730.0
2,3,927.0,1.0,135293,21167.0
3,3,21.0,5.0,166998,18528.0
4,3,4475.0,5.0,53504,26989.0
...,...,...,...,...,...
141443,4,4681.0,4.0,152060,2469.0
141444,4,5123.0,3.0,89235,18998.0
141445,6,435.0,5.0,119788,26989.0
141446,5,176.0,3.0,86824,10193.0


In [28]:
y_ = xgbr.predict(x_)
y_

array([106.62639 ,  63.74353 ,  40.119846, ...,  58.68849 ,  17.077082,
        26.31199 ], dtype=float32)

In [29]:
df_main['Upvotes'] = abs(y_)
df_main

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,366953,0,5123.0,3.0,50652,26989.0,106.626389
1,71864,1,5123.0,5.0,37685,2730.0,63.743530
2,141692,3,927.0,1.0,135293,21167.0,40.119846
3,316833,3,21.0,5.0,166998,18528.0,12.128903
4,440445,3,4475.0,5.0,53504,26989.0,105.809608
...,...,...,...,...,...,...,...
141443,47187,4,4681.0,4.0,152060,2469.0,23.822334
141444,329126,4,5123.0,3.0,89235,18998.0,105.157318
141445,282334,6,435.0,5.0,119788,26989.0,58.688492
141446,386629,5,176.0,3.0,86824,10193.0,17.077082


In [30]:
sample = df_main[["ID","Upvotes"]]
sample = sample[["ID","Upvotes"]].apply(np.floor)
sample

,ID,Upvotes
0,366953.0,106.0
1,71864.0,63.0
2,141692.0,40.0
3,316833.0,12.0
4,440445.0,105.0
...,...,...
141443,47187.0,23.0
141444,329126.0,105.0
141445,282334.0,58.0
141446,386629.0,17.0


In [31]:
asc = sample.sort_values(by=['ID'], ascending=True)
asc.to_csv("output_xgbr.csv", index=False)